In [1]:
import sys
sys.path.append("../")
from plotting import CandlePlot
import pandas as pd
from technical.patterns import apply_patterns

In [2]:
df = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [3]:
df_an = df[["time", "mid_o", "mid_h","mid_l", "mid_c" ]].copy()

In [4]:
df_an.tail()

,time,mid_o,mid_h,mid_l,mid_c
3996,2025-05-09 16:00:00+00:00,193.280,193.392,193.222,193.326
3997,2025-05-09 17:00:00+00:00,193.322,193.469,193.291,193.385
3998,2025-05-09 18:00:00+00:00,193.384,193.438,193.274,193.424
3999,2025-05-09 19:00:00+00:00,193.416,193.460,193.336,193.422
4000,2025-05-09 20:00:00+00:00,193.415,193.455,193.282,193.408


In [9]:
direction = df_an.mid_c - df_an.mid_o 
body_size = abs(direction)
direction = [1 if x>= 0 else -1 for x in direction]
full_range = df_an.mid_h - df_an.mid_l
body_perc = (body_size / full_range) * 100 
body_lower = df_an[["mid_c", "mid_o"]].min(axis=1)
body_upper = df_an[["mid_c", "mid_o"]].max(axis=1)
body_bottom_perc =( body_lower - df_an.mid_l) / full_range * 100
body_top_perc = 100 - (((df_an.mid_h - body_upper) / full_range) * 100)

In [10]:
df_an['body_lower'] = body_lower
df_an["body_upper"] = body_upper
df_an['body_bottom_perc'] = body_bottom_perc
df_an["body_top_perc"] = body_top_perc
df_an['body_per'] = body_perc

In [5]:
df_an = apply_patterns(df_an)

In [6]:
dfp = df_an.iloc[-50:]
cp = CandlePlot(dfp)
cp.show_plot(line_traces=[])

In [14]:
df_an.iloc[-6]

time                2025-05-09 15:00:00+00:00
mid_o                                 192.921
mid_h                                 193.282
mid_l                                 192.908
mid_c                                 193.282
body_lower                            192.921
body_upper                            193.282
body_bottom_perc                     3.475936
body_top_perc                           100.0
body_per                            96.524064
Name: 3995, dtype: object